## Round 1 
* https://github.com/cal-itp/data-analyses/issues/1059
* cd rt_segment_speeds && pip install altair_transform && pip install -r requirements.txt && cd ../_shared_utils && make setup_env
* https://docs.google.com/document/d/1I1WiqlmU06W6iLCi7cZQrOCLILkrEfABEkcU0Jys7f0/edit
* https://route-speeds--cal-itp-data-analyses.netlify.app/name_bay-area-511-muni-schedule/0__report__name_bay-area-511-muni-schedule
* https://posit-dev.github.io/great-tables/get-started/nanoplots.html
* https://docs.pola.rs/py-polars/html/reference/api/polars.from_pandas.html
* https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/_rt_scheduled_utils.py
* https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/_threshold_utils.py

In [1]:
%%capture
# import warnings
# warnings.filterwarnings('ignore')

import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import great_tables as gt
import pandas as pd
from calitp_data_analysis import calitp_color_palette as cp
from great_tables import md
from IPython.display import HTML, Markdown, display
from segment_speed_utils.project_vars import RT_SCHED_GCS
from shared_utils import rt_dates, rt_utils

alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)
from typing import List, Union

from altair_transform.extract import extract_transform
from altair_transform.transform import visit
from altair_transform.utils import to_dataframe

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
name = "SBMTD Schedule"

In [4]:
# %%capture_parameters
# name

### General Functions

In [5]:
def labeling(word: str) -> str:
    return word.replace("_", " ").title().replace("N", "Total").replace("Pct", "%")

### Data

In [6]:
# calitp-analytics-data/data-analyses/rt_vs_schedule/digest
df = pd.read_parquet(
    f"{RT_SCHED_GCS}digest/schedule_vp_metrics.parquet",
    filters=[[("name", "==", name)]],
)

In [7]:
f"{RT_SCHED_GCS}digest/schedule_vp_metrics.parquet"

'gs://calitp-analytics-data/data-analyses/rt_vs_schedule/digest/schedule_vp_metrics.parquet'

In [8]:
most_recent_date = df.service_date.max()

In [9]:
most_recent_date

Timestamp('2024-03-13 00:00:00')

In [10]:
df.service_date.min()

Timestamp('2023-03-15 00:00:00')

#### Unsure what these metrics mean
* https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/segment_speed_utils/metrics.py
* avg_stop_miles: ??
* n_scheduled_trips (over the course of a day?)
* frequency: # of times the route runs per hour 
* pct_typology: 44% of the route fits into downtownlocal, the rest of the 54% falls under other categories.
* is_early: % of trips in that route htat is early
* is_on_time: % of trips that are ontime.
* How come the % of is_late, is_ontime, and is_early don't add up?
* n_vp_trips: ??
* Difference between pct_rt_journey_atleast1vp and pct_sched_journey_atleast1vp?
* rt_sched_journey_ratio: how many times longer/shorter the actual trip took compared to the scheduled trip?
* avg_rt_servie_minutes?

In [11]:
900.19 / 795

1.132314465408805

In [12]:
df.head(2)

,schedule_gtfs_dataset_key,direction_id,time_period,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,road_freq_category,road_typology,pct_typology,service_date,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,sched_rt_category,speed_mph,name,route_id,route_combined_name,base64_url,organization_source_record_id,organization_name,caltrans_district
14844,239f3baf3dd3b9e9464f66a777f9897d,0.00,all_day,15.00,0.18,54,2.25,very_high,downtown_local,0.44,2023-09-13,863,258,900.19,795.00,1136,925,0,45,8,53,1.26,0.81,0.96,0.29,1.00,0.33,1.13,16.98,schedule_and_vp,8.92,SBMTD Schedule,1,1 West Santa Barbara,aHR0cHM6Ly9zYm10ZC5nb3YvZ29vZ2xlX3RyYW5zaXQvZmVlZC56aXA=,recswCrw6a6htmXJ4,Santa Barbara Metropolitan Transit District,05 - San Luis Obispo
14845,239f3baf3dd3b9e9464f66a777f9897d,0.00,all_day,15.00,0.18,54,2.25,very_high,downtown_local,0.44,2023-10-11,839,242,945.11,735.00,1093,848,0,38,11,49,1.16,0.78,0.89,0.26,1.00,0.33,1.29,19.29,schedule_and_vp,4.23,SBMTD Schedule,1,1 West Santa Barbara,aHR0cHM6Ly9zYm10ZC5nb3YvZ29vZ2xlX3RyYW5zaXQvZmVlZC56aXA=,recswCrw6a6htmXJ4,Santa Barbara Metropolitan Transit District,05 - San Luis Obispo


In [13]:
df.time_period.value_counts()

all_day    731
peak       731
offpeak    451
Name: time_period, dtype: int64

### Test out Altair `extract_data`
* https://altair-viz.github.io/user_guide/transform/index.html

In [14]:
__all__ = ["apply", "extract_data", "transform_chart"]

In [15]:
def reverse_snakecase(df):
    """
    Clean up columns to remove underscores and spaces.
    """
    df.columns = df.columns.str.replace("_", " ").str.strip().str.title()
    return df

In [16]:
def apply(
    df: pd.DataFrame,
    transform: Union[alt.Transform, List[alt.Transform]],
    inplace: bool = False,
) -> pd.DataFrame:
    """Apply transform or transforms to dataframe.

    Parameters
    ----------
    df : pd.DataFrame
    transform : list|dict
        A transform specification or list of transform specifications.
        Each specification must be valid according to Altair's transform
        schema.
    inplace : bool
        If True, then dataframe may be modified in-place. Default: False.

    Returns
    -------
    df_transformed : pd.DataFrame
        The transformed dataframe.

    Example
    -------
    >>> import pandas as pd
    >>> data = pd.DataFrame({'x': range(5), 'y': list('ABCAB')})
    >>> chart = alt.Chart(data).transform_aggregate(sum_x='sum(x)', groupby=['y'])
    >>> apply(data, chart.transform)
       y  sum_x
    0  A      3
    1  B      5
    2  C      2
    """
    if not inplace:
        df = df.copy()
    if transform is alt.Undefined:
        return df
    return visit(transform, df)

In [17]:
def extract_data(
    chart: alt.Chart, apply_encoding_transforms: bool = True
) -> pd.DataFrame:
    """Extract transformed data from a chart.

    This only works with data and transform defined at the
    top level of the chart.

    Parameters
    ----------
    chart : alt.Chart
        The chart instance from which the data and transform
        will be extracted
    apply_encoding_transforms : bool
        If True (default), then apply transforms specified within an
        encoding as well as those specified directly in the transforms
        attribute.

    Returns
    -------
    df_transformed : pd.DataFrame
        The extracted and transformed dataframe.

    Example
    -------
    >>> import pandas as pd
    >>> data = pd.DataFrame({'x': range(5), 'y': list('ABCAB')})
    >>> chart = alt.Chart(data).mark_bar().encode(x='sum(x)', y='y')
    >>> extract_data(chart)
       y  sum_x
    0  A      3
    1  B      5
    2  C      2
    """
    if apply_encoding_transforms:
        chart = extract_transform(chart)
    return apply(to_dataframe(chart.data, chart), chart.transform)

In [18]:
data = pd.DataFrame({"x": range(5), "y": list("ABCAB")})

In [19]:
chart = alt.Chart(data).mark_bar().encode(x="sum(x)", y="y")

In [20]:
chart

alt.Chart(...)

In [21]:
extract_data(chart)

,y,sum_x
0,A,3
1,B,5
2,C,2


In [22]:
def extract_data_altair(chart):
    chart_dict = chart.to_dict()
    encoding = chart_dict["datasets"]
    df1 = pd.DataFrame(encoding)

    column = df1.columns[0]
    normalized_df = pd.json_normalize(df1[column])
    # Combine the original DataFrame with the extracted values DataFrame
    df2 = pd.concat([df1.drop(column, axis=1), normalized_df], axis=1)
    return df2

### Monthly aggregated service hours by day_type, time_of_day

In [23]:
from segment_speed_utils.project_vars import SCHED_GCS

In [24]:
year = "2023"

In [25]:
monthly_service_df = pd.read_parquet(
    f"{SCHED_GCS}scheduled_service_by_route_{year}.parquet",
    filters=[[("name", "==", name)]],
)

In [26]:
monthly_service_df.shape

(8007, 12)

In [27]:
monthly_service_df.sample()

,key,name,source_record_id,route_id,route_short_name,route_long_name,time_of_day,month,year,day_type,n_trips,ttl_service_hours
1604,ffc0dc963521f9f51b2730eb9061617f,SBMTD Schedule,rectQfIeiKDBeJSAV,6,6,Goleta,Midday,4,2023,1,100,87.50


In [28]:
monthly_service_df["full_date"] = pd.to_datetime(
    (monthly_service_df.month.astype(str) + "-" + monthly_service_df.year.astype(str))
)

In [29]:
def tag_day(df: pd.DataFrame) -> pd.DataFrame:
    # Function to determine if a date is a weekend day or a weekday
    def which_day(date):
        if date == 1:
            return "Monday"
        elif date == 2:
            return "Tuesday"
        elif date == 3:
            return "Wednesday"
        elif date == 4:
            return "Thursday"
        elif date == 5:
            return "Friday"
        elif date == 6:
            return "Saturday"
        else:
            return "Sunday"

    # Apply the function to each value in the "service_date" column
    df["day_type"] = df["day_type"].apply(which_day)

    return df

In [30]:
monthly_service_df = tag_day(monthly_service_df)

In [31]:
monthly_service = (
    monthly_service_df.groupby(
        [
            "full_date",
            "month",
            "name",
            "day_type",
            "time_of_day",
        ]
    )
    .agg(
        {
            "ttl_service_hours": "mean",
        }
    )
    .reset_index()
)

####  Fix
* Dropdown menu should have the fully spelled months?
* Dropdown menu does not like datetime values, pad single digit months with a 0 and concat it with the year?

In [32]:
def bar_chart_dropdown(
    df: pd.DataFrame,
    x_col: str,
    y_col: str,
    offset_col: str,
    title: str,
    dropdown_col: str,
):
    dropdown_list = sorted(df[dropdown_col].unique().tolist())

    initialize_first_val = sorted(dropdown_list)[0]

    dropdown = alt.binding_select(options=dropdown_list, name=labeling(dropdown_col))

    selector = alt.selection_single(
        name=labeling(dropdown_col), fields=[dropdown_col], bind=dropdown
    )

    ruler = (
        alt.Chart(df)
        .mark_rule(color="red", strokeDash=[10, 7])
        .encode(y=f"mean({y_col}):Q")
    )

    chart = (
        alt.Chart(monthly_service)
        .mark_bar()
        .encode(
            x=alt.X(
                f"{x_col}:N",
                title="Day",
                scale=alt.Scale(
                    domain=[
                        "Monday",
                        "Tuesday",
                        "Wednesday",
                        "Thursday",
                        "Friday",
                        "Saturday",
                        "Sunday",
                    ]
                ),
            ),
            y=alt.Y(f"{y_col}:Q", title=labeling(y_col)),
            xOffset=f"{offset_col}:N",
            color=alt.Color(
                f"{offset_col}:N",
                scale=alt.Scale(
                    range=cp.CALITP_SEQUENTIAL_COLORS,
                ),
            ),
            tooltip=df.columns.tolist(),
        )
    )
    chart = (chart + ruler).properties(title=title, width=600, height=400)
    chart = chart.add_params(selector).transform_filter(selector)
    # data = chart.transformed_data()

    display(chart)

In [33]:
bar_chart_dropdown(
    monthly_service,
    "day_type",
    "ttl_service_hours",
    "time_of_day",
    "Average Total Service Hours across all Routes in 2023",
    "month",
)

/opt/conda/lib/python3.9/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.LayerChart(...)

### Monthly Trends
* https://posit-dev.github.io/great-tables/articles/intro.html

In [34]:
by_date_category = (
    pd.crosstab(
        df.service_date,
        df.sched_rt_category,
        values=df.n_scheduled_trips,
        aggfunc="sum",
    )
    .reset_index()
    .fillna(0)
)

In [35]:
(
    gt.GT(by_date_category, rowname_col="service_date")
    .tab_header(
        title="Daily Trips by GTFS Availability",
        subtitle="Schedule only indicates the trip(s) were found only in schedule data. Vehicle Positions (VP) only indicates the trip(s) were found only in real-time data.",
    )
    .cols_label(
        schedule_only="Schedule Only",
        vp_only="VP Only",
        schedule_and_vp="Schedule and VP",
    )
    .fmt_integer(["schedule_only", "vp_only", "schedule_and_vp"])
    .tab_options(container_width="100%")
    .tab_options(table_font_size="12px")
)

GT(_tbl_data=sched_rt_category service_date  schedule_only  vp_only  schedule_and_vp
0                   2023-03-15              4        0             1672
1                   2023-04-12              3        0             1673
2                   2023-05-17             18        0             1658
3                   2023-06-14              2        0             1604
4                   2023-07-12              0        0             1598
5                   2023-08-15              0        0             1612
6                   2023-09-13             14        0             1684
7                   2023-10-11             12        0             1716
8                   2023-12-13             18        0             1710
9                   2024-01-17             14        0             1702
10                  2024-02-14             18        0             1710
11                  2024-03-13             15        0             1713, _body=<great_tables._gt_data.Body object at 0x7f79164ac8e0>, _boxhead=Boxhead([ColInfo(var='service_date', type=<ColInfoTypeEnum.stub: 2>, column_label='service_date', column_align='right', column_width=None), ColInfo(var='schedule_only', type=<ColInfoTypeEnum.default: 1>, column_label='Schedule Only', column_align='right', column_width=None), ColInfo(var='vp_only', type=<ColInfoTypeEnum.default: 1>, column_label='VP Only', column_align='right', column_width=None), ColInfo(var='schedule_and_vp', type=<ColInfoTypeEnum.default: 1>, column_label='Schedule and VP', column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=Timestamp('2023-03-15 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=Timestamp('2023-04-12 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=Timestamp('2023-05-17 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=Timestamp('2023-06-14 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=Timestamp('2023-07-12 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=5, group_id=None, rowname=Timestamp('2023-08-15 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=6, group_id=None, rowname=Timestamp('2023-09-13 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=7, group_id=None, rowname=Timestamp('2023-10-11 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=8, group_id=None, rowname=Timestamp('2023-12-13 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=9, group_id=None, rowname=Timestamp('2024-01-17 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=10, group_id=None, rowname=Timestamp('2024-02-14 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=11, group_id=None, rowname=Timestamp('2024-03-13 00:00:00'), group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([]), _heading=Heading(title='Daily Trips by GTFS Availability', subtitle='Schedule only indicates the trip(s) were found only in schedule data. Vehicle Positions (VP) only indicates the trip(s) were found only in real-time data.', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x7f79164e6700>, _formats=[<great_tables._gt_data.FormatInfo object at 0x7f79164e69d0>], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF')

#### GTFS Availability
* Change Category values to something more understandable?

In [36]:
route_categories = (
    df[df.time_period == "all_day"]
    .groupby("sched_rt_category")
    .agg({"route_combined_name": "nunique"})
    .reset_index()
)

In [37]:
route_categories.sched_rt_category = route_categories.sched_rt_category.str.replace(
    "_", " "
).str.title()

In [38]:
(
    gt.GT(data=route_categories.dropna())
    .fmt_integer(columns=["route_combined_name"], compact=True)
    .cols_label(route_combined_name="Total Routes", sched_rt_category="Category")
    .tab_options(container_width="100%")
    .tab_header(
        title="Routes with GTFS Availability",
        subtitle="Schedule only indicates the route(s) were found only in static, schedule data. Vehicle Positions (VP) only indicates the route(s) were found only in real-time data.",
    )
    .tab_options(table_font_size="12px")
)

GT(_tbl_data=  sched_rt_category  route_combined_name
0     Schedule Only                   16
1           Vp Only                    3
2   Schedule And Vp                   35, _body=<great_tables._gt_data.Body object at 0x7f79164a2760>, _boxhead=Boxhead([ColInfo(var='sched_rt_category', type=<ColInfoTypeEnum.default: 1>, column_label='Category', column_align='left', column_width=None), ColInfo(var='route_combined_name', type=<ColInfoTypeEnum.default: 1>, column_label='Total Routes', column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([]), _heading=Heading(title='Routes with GTFS Availability', subtitle='Schedule only indicates the route(s) were found only in static, schedule data. Vehicle Positions (VP) only indicates the route(s) were found only in real-time data.', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x7f791649ed90>, _formats=[<great_tables._gt_data.FormatInfo object at 0x7f791636f430>], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='12px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading'

#### Route Stats

In [39]:
def route_stats(df: pd.DataFrame) -> pd.DataFrame:
    route_merge_cols = ["route_combined_name", "direction_id"]

    all_day_stats = df[
        (df.service_date == most_recent_date) & (df.time_period == "all_day")
    ][
        route_merge_cols
        + [
            "avg_scheduled_service_minutes",
            "avg_stop_miles",
            "n_scheduled_trips",
            "sched_rt_category",
        ]
    ]

    peak_stats = df[(df.service_date == most_recent_date) & (df.time_period == "peak")][
        route_merge_cols + ["speed_mph", "n_scheduled_trips", "frequency"]
    ].rename(
        columns={
            "speed_mph": "peak_avg_speed",
            "n_scheduled_trips": "peak_scheduled_trips",
            "frequency": "peak_hourly_freq",
        }
    )

    offpeak_stats = df[
        (df.service_date == most_recent_date) & (df.time_period == "offpeak")
    ][route_merge_cols + ["speed_mph", "n_scheduled_trips", "frequency"]].rename(
        columns={
            "speed_mph": "offpeak_avg_speed",
            "n_scheduled_trips": "offpeak_scheduled_trips",
            "frequency": "offpeak_hourly_freq",
        }
    )

    table_df = (
        pd.merge(
            all_day_stats,
            peak_stats,
            on=route_merge_cols,
        )
        .merge(offpeak_stats, on=route_merge_cols)
        .sort_values(["route_combined_name", "direction_id"])
        .reset_index(drop=True)
    )

    numeric_cols = table_df.select_dtypes(include="number").columns
    table_df[numeric_cols] = table_df[numeric_cols].fillna(0)

    return table_df

In [40]:
table_df = route_stats(df)

### Updating Already Made Charts
* checking out another ways to display metrics, which ones can be cut.

In [41]:
sched_df = df[df.sched_rt_category != "vp_only"]
vp_df = df[df.sched_rt_category != "schedule_only"]

sched_service_chart = sched_df[sched_df.time_period == "all_day"]

#### Graph Functions
* What if a subtitle needs to be broken up?

In [42]:
def grouped_bar_chart(
    df: pd.DataFrame,
    color_col: str,
    y_col: str,
    offset_col: str,
    title: str,
    subtitle: str,
):
    df = df.assign(
        time_period=df.time_period.str.replace("_", " ").str.title()
    ).reset_index(drop=True)

    df[y_col] = df[y_col].fillna(0).astype(int)
    tooltip_cols = [
        "direction_id",
        "time_period",
        "route_combined_name",
        "organization_name",
        "service_date",
        y_col,
    ]
    """ 
    ruler = (
        alt.Chart(df)
        .mark_rule(color="red", strokeDash=[10, 7])
        .encode(y=f"mean({y_col}):Q")
    )
    """
    chart = (
        alt.Chart(df)
        .mark_bar(size=10)
        .encode(
            x=alt.X(
                "yearmonthdate(service_date):O",
                title=["Grouped by Direction ID", "Date"],
                axis=alt.Axis(format="%b %Y"),
            ),
            y=alt.Y(f"{y_col}:Q", title=labeling(y_col)),
            xOffset=alt.X(f"{offset_col}:N", title=labeling(offset_col)),
            color=alt.Color(
                f"{color_col}:N",
                title=labeling(color_col),
                scale=alt.Scale(
                    range=cp.CALITP_SEQUENTIAL_COLORS,
                ),
            ),
            tooltip=tooltip_cols,
        )
    )
    chart = (chart).properties(
        title={
            "text": [title],
            "subtitle": [subtitle],
        },
        width=500,
        height=300,
    )

    return chart

In [43]:
def heatmap(
    df: pd.DataFrame,
    color_col: str,
    title: str,
    subtitle1: str,
    subtitle2: str,
    subtitle3: str,
):
    df = df.assign(
        time_period=df.time_period.str.replace("_", " ").str.title()
    ).reset_index(drop=True)

    tooltip_cols = [
        "direction_id",
        "time_period",
        "route_combined_name",
        "organization_name",
        color_col,
    ]

    # Round
    df[color_col] = df[color_col].round(1)
    chart = (
        alt.Chart(df)
        .mark_rect(size=30)
        .encode(
            x=alt.X(
                "yearmonthdate(service_date):O",
                axis=alt.Axis(labelAngle=-45, format="%b %Y"),
                title=["Grouped by Direction ID", "Service Date"],
            ),
            y=alt.Y("time_period:O", title=["Time Period"]),
            xOffset=alt.X(f"direction_id:N", title="Direction ID"),
            color=alt.Color(
                f"{color_col}:Q",
                title=labeling(color_col),
                scale=alt.Scale(range=cp.CALITP_SEQUENTIAL_COLORS),
            ),
            tooltip=tooltip_cols,
        )
        .properties(
            title={"text": [title], "subtitle": [subtitle1, subtitle2, subtitle3]},
            width=500,
            height=300,
        )
    )

    text = chart.mark_text(baseline="middle").encode(
        alt.Text("direction_id"), color=alt.value("white")
    )

    final_chart = chart + text
    return final_chart

In [44]:
def base_route_chart_ah(
    df: pd.DataFrame, y_col: str, title: str, subtitle: str
) -> alt.Chart:

    df = df.assign(
        time_period=df.time_period.str.replace("_", " ").str.title()
    ).reset_index(drop=True)

    selected_colors = [
        cp.CALITP_CATEGORY_BRIGHT_COLORS[0],  # blue
        cp.CALITP_CATEGORY_BRIGHT_COLORS[3],  # green
        cp.CALITP_CATEGORY_BOLD_COLORS[1],  # orange,
    ]

    # https://stackoverflow.com/questions/26454649/python-round-up-to-the-nearest-ten

    if "pct" in y_col:
        max_y = 1.2
    elif "per_minute" in y_col:
        max_y = round(df[y_col].max())
    else:
        max_y = round(df[y_col].max(), -1) + 5

    df[f"{y_col}_str"] = df[y_col].astype(str)

    ruler = (
        alt.Chart(df)
        .mark_rule(color="red", strokeDash=[10, 7])
        .encode(y=f"mean(speed_mph):Q")
    )

    chart = (
        alt.Chart(df)
        .mark_line(size=5)
        .encode(
            x=alt.X(
                "yearmonthdate(service_date):O",
                title="Date",
                axis=alt.Axis(format="%b %Y"),
            ),
            y=alt.Y(
                f"{y_col}:Q", title=labeling(y_col), scale=alt.Scale(domain=[0, max_y])
            ),
            color=alt.Color(
                "time_period:N",
                title=labeling("time_period"),
                scale=alt.Scale(range=cp.CALITP_SEQUENTIAL_COLORS),
            ),
            tooltip=[
                "route_combined_name",
                "route_id",
                "direction_id",
                "time_period",
                f"{y_col}_str",
            ],
        )
    )

    chart = (chart + ruler).properties(width=250, height=300)
    chart = chart.facet(
        column=alt.Column("direction_id:N", title=labeling("direction_id")),
    ).properties(
        title={
            "text": [title],
            "subtitle": [subtitle],
        },
    )
    return chart

In [45]:
one_route = sched_df.loc[sched_df.route_combined_name == "1 West Santa Barbara"]

#### Avg Scheduled Minutes'
* How come the average_scheduled_minutes is a lot shorter than total_rt_service_minutes and total_scheduled_service_minutes?
* Kind of a boring chart? Should compare actual run time compared to service minutes?

In [46]:
# one_route[one_route.time_period == "all_day"][['avg_scheduled_service_minutes']]

In [47]:
grouped_bar_chart(
    df=one_route[one_route.time_period == "all_day"],
    color_col="direction_id",
    y_col="avg_scheduled_service_minutes",
    offset_col="direction_id",
    title="Average Scheduled Minutes",
    subtitle="The average minutes a route is scheduled to run.",
)

alt.Chart(...)

#### Timeliness #1

In [48]:
one_route[one_route.time_period == "all_day"].head(1)

,schedule_gtfs_dataset_key,direction_id,time_period,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,road_freq_category,road_typology,pct_typology,service_date,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,sched_rt_category,speed_mph,name,route_id,route_combined_name,base64_url,organization_source_record_id,organization_name,caltrans_district
14844,239f3baf3dd3b9e9464f66a777f9897d,0.00,all_day,15.00,0.18,54,2.25,very_high,downtown_local,0.44,2023-09-13,863,258,900.19,795.00,1136,925,0,45,8,53,1.26,0.81,0.96,0.29,1.00,0.33,1.13,16.98,schedule_and_vp,8.92,SBMTD Schedule,1,1 West Santa Barbara,aHR0cHM6Ly9zYm10ZC5nb3YvZ29vZ2xlX3RyYW5zaXQvZmVlZC56aXA=,recswCrw6a6htmXJ4,Santa Barbara Metropolitan Transit District,05 - San Luis Obispo


In [49]:
avg_rt_service_minutes = 16.98

In [50]:
avg_scheduled_service_minutes = 15

In [51]:
avg_rt_service_minutes / avg_scheduled_service_minutes

1.1320000000000001

In [52]:
rt_v_scheduled_caption = """Dividing the average of actual trip times by the average of scheduled trip
times gives an idea of how on schedule a trip on this route typically is. A ratio of 1.13 indicates a trip is late 13% over its scheduled time."""

In [53]:
heatmap(
    one_route,
    "rt_sched_journey_ratio",
    "Realtime vs. Scheduled Trip Times",
    "Dividing the average of actual trip times by the average of scheduled trip times gives an idea of how on schedule a trip on this route typically is.",
    "A ratio of 1.13 indicates a trip is late 13% over its scheduled time.",
    "",
)

alt.LayerChart(...)

#### Timeliness 2

In [54]:
one_route.sample()

,schedule_gtfs_dataset_key,direction_id,time_period,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,road_freq_category,road_typology,pct_typology,service_date,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,sched_rt_category,speed_mph,name,route_id,route_combined_name,base64_url,organization_source_record_id,organization_name,caltrans_district
105801,de792182088eecc3d5c0bd3f1df62965,0.00,offpeak,15.00,0.18,27,1.69,very_high,downtown_local,0.42,2023-05-17,416,119,428.45,390.00,537,433,0,24,2,26,1.25,0.81,0.97,0.28,1.00,0.30,1.10,16.48,schedule_and_vp,8.68,SBMTD Schedule,1,1 West Santa Barbara,aHR0cHM6Ly9zYm10ZC5nb3YvZ29vZ2xlX3RyYW5zaXQvZmVlZC56aXA=,recswCrw6a6htmXJ4,Santa Barbara Metropolitan Transit District,05 - San Luis Obispo


In [55]:
timeliness2 = (
    one_route.groupby(["service_date", "route_combined_name", "time_period"])
    .agg({"avg_rt_service_minutes": "max", "avg_scheduled_service_minutes": "max"})
    .reset_index()
)

In [56]:
timeliness3 = timeliness2.melt(
    id_vars=["service_date", "route_combined_name", "time_period"],
    value_vars=["avg_rt_service_minutes", "avg_scheduled_service_minutes"],
)

In [57]:
timeliness3.head()

,service_date,route_combined_name,time_period,variable,value
0,2023-03-15,1 West Santa Barbara,all_day,avg_rt_service_minutes,18.14
1,2023-03-15,1 West Santa Barbara,offpeak,avg_rt_service_minutes,17.07
2,2023-03-15,1 West Santa Barbara,peak,avg_rt_service_minutes,19.20
3,2023-04-12,1 West Santa Barbara,all_day,avg_rt_service_minutes,18.60
4,2023-04-12,1 West Santa Barbara,offpeak,avg_rt_service_minutes,17.89


In [74]:
(
    alt.Chart(timeliness3)
    .mark_line(size=5)
    .encode(
        x=alt.X(
            "yearmonthdate(service_date):O",
            title="Date",
            axis=alt.Axis(format="%b %Y"),
        ),
        y=alt.Y(
            "value:Q",
            title=labeling("value"),
            scale=alt.Scale(domain=[(timeliness3.value.min()-5), (timeliness3.value.max()+5)]),
        ),
        color=alt.Color(
            "variable:N",
            title=labeling("variable"),
            scale=alt.Scale(range=cp.CALITP_SEQUENTIAL_COLORS),
        ),
        tooltip=timeliness3.columns.tolist(),
    )
).properties(width=150, height=300).facet(
    column=alt.Column("time_period:N", title=labeling("direction_id")),
)

alt.FacetChart(...)

In [ ]:
chart = (chart).properties(
    title={
        "text": [title],
        "subtitle": [subtitle],
    },
    width=500,
    height=300,
)

In [ ]:
chart = (chart + ruler).properties(width=250, height=300)
chart = chart.facet(
    column=alt.Column("direction_id:N", title=labeling("direction_id")),
).properties(
    title={
        "text": [title],
        "subtitle": [subtitle],
    },
)

#### Total Scheduled Trips
* Maybe don't differentiate this by direction_id??
* Kind of confusing because it's trip for only one direction...
* Similar to frequency already?

In [ ]:
110 / 24

In [ ]:
# IF we take away direction, see how many times a route is scheduled to run
total_scheduled_trip = (
    one_route.groupby(["service_date", "time_period"])
    .agg({"n_scheduled_trips": "mean"})
    .reset_index()
)

In [ ]:
total_scheduled_trip.n_scheduled_trips = total_scheduled_trip.n_scheduled_trips / 2

In [ ]:
total_scheduled_trip.head()

In [ ]:
one_route.groupby(["service_date", "direction_id", "time_period"]).agg(
    {"n_scheduled_trips": "max"}
).head()

In [ ]:
grouped_bar_chart(
    df=one_route.loc[one_route.time_period != "all_day"],
    color_col="time_period",
    y_col="n_scheduled_trips",
    offset_col="direction_id",
    title="Total Scheduled Trips",
    subtitle="Displays how many times per day is this route is scheduled to run in one particular direction.",
)

#### Frequency
* Maybe shouldn't be a chart since there doesn't seem to be a lot of data for this across a lot of the routes?
* What does frequency mean?
* Simplify down to not take direction_id into consideration?

In [ ]:
heatmap(
    one_route,
    "frequency",
    "Frequency of Route",
    "Frequency tracks the number of times per hour this route goes by direction and time period.",
    "For example, a frequency of 2.3 going in the direction of ID 1 means the bus passes by this direction about",
    "twice an hour.",
)

#### Speed MPH
* Needs a different type of chart.

In [ ]:
one_route_vp_df = vp_df.loc[vp_df.route_combined_name == "1 West Santa Barbara"]

In [ ]:
one_route_vp_df.shape

In [ ]:
one_route_vp_df[
    ["direction_id", "time_period", "service_date", "speed_mph"]
].sort_values(by=["service_date", "time_period"]).head()

In [ ]:
base_route_chart_ah(
    one_route_vp_df,
    "speed_mph",
    "Average Speed",
    "The average miles per hour the bus travels by direction and time of day.",
)

#### VP per Minute (WIP)
* Should be a table?

In [ ]:
heatmap(
    one_route_vp_df,
    "vp_per_minute",
    "Vehicle Positions per Minute",
    "Vehicle positions are recorded each time the GPS on a bus pings.",
    "The number of times vehicle positions collected reflect the density and reliabilty of realtime data",
    "Generally, the GPS should record vehicle positions every 20 seconds.",
)

#### Spatial Accuracy
* Multiple it by 100? Should this be rounded or not?

In [ ]:
heatmap(
    one_route_vp_df,
    "pct_in_shape",
    "Spatial Accuracy",
    "By comparing vehicle positions produced by GTFS and against the static scheduled route shape,",
    " we can determine the percentage of actual positions that fell within a reasonable radius.",
    "This percentage reflects the accuracy of the spatial, realtime data collected.",
)

#### At least 2 VP

In [ ]:
one_route_vp_df[["pct_rt_journey_atleast2_vp"]].sample()

In [ ]:
heatmap(
    one_route_vp_df,
    "pct_rt_journey_atleast2_vp",
    "% of the Trip with 2+ Vehicle Positions",
    "Dividing the number of minutes with more than 2 vehicle positions by the total duration",
    "to determine the quality of real-time temporal data for a trip. Generally, the GPS should record vehicle positions every 20 seconds.",
    "A percentage of 0.3 indicates only 30% of the trip had 2+ vehicle positions per minute.",
)

#### Text

In [ ]:
text_one_route = table_df.loc[table_df.route_combined_name == "14 Montecito"]

In [ ]:
text_one_route.sample()

In [ ]:
def create_text_table(df: pd.DataFrame):
    to_keep = [
        "route_combined_name",
        "sched_rt_category",
        "avg_stop_miles",
        "peak_scheduled_trips",
        "offpeak_scheduled_trips",
        "direction_id",
        "peak_avg_speed",
        "offpeak_avg_speed",
        "peak_hourly_freq",
        "offpeak_hourly_freq",
    ]

    df2 = df[to_keep]
    df2 = reverse_snakecase(df2)

    # Transform and rename columns
    df2 = df2.T.reset_index()
    df2.columns.values[1] = "Direction ID 0"
    df2.columns.values[2] = "Direction ID 1"

    # Create a decoy column to center all the text
    df2["Zero"] = 0

    text_chart = (
        alt.Chart(df2)
        .mark_text()
        .encode(x=alt.X("Zero:Q", axis=None), y=alt.Y("index", axis=None))
        .properties(title="Route Statistics", width=500, height=300)
    )

    # Create charts
    columns_chart = text_chart.encode(text="index:N").properties(
        title="Columns", width=10
    )
    dir0_chart = text_chart.encode(text="Direction ID 0:N").properties(
        title="Direction 0", width=10
    )

    dir1_chart = text_chart.encode(text="Direction ID 1:N").properties(
        title="Direction 1", width=10
    )

    # Combine data tables
    text = alt.hconcat(columns_chart, dir0_chart, dir1_chart)

    return text

In [ ]:
table_df_test = table_df.T.reset_index()

In [ ]:
table_df_test.head()

In [ ]:
table_df.head(2)

#### Putting it all together

In [ ]:
def filtered_route(
    df: pd.DataFrame,
) -> alt.Chart:
    """
    https://stackoverflow.com/questions/58919888/multiple-selections-in-altair
    """

    route_dropdown = alt.binding_select(
        options=sorted(df["route_combined_name"].unique().tolist()),
        name="Routes ",
    )

    # Column that controls the bar charts
    route_selector = alt.selection_point(
        fields=["route_combined_name"],
        bind=route_dropdown,
    )

    sched_df = df[df.sched_rt_category != "vp_only"]
    vp_df = df[df.sched_rt_category != "schedule_only"]

    # route_stats_df = route_stats(df)
    # route_stats_chart = (create_text_table(route_stats_df).add_params(route_selector)
    #    .transform_filter(route_selector))

    speeds_chart = (
        base_route_chart_ah(vp_df, "speed_mph", "Speeds MPH", "my_subtitle")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    ping_density_chart = (
        heatmap(vp_df, "vp_per_minute", "Vehicle Positions per Minute", "subtitle")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    atleast2vp_chart = (
        heatmap(
            vp_df,
            "pct_rt_journey_atleast2_vp",
            "% of RT Trips with at least 2 Vehicle Positions",
            "Lorem ipsum dolor sit amet, consectetur adipiscing elit",
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    chart_lists = [speeds_chart, ping_density_chart, atleast2vp_chart]
    chart = alt.vconcat(*chart_lists).resolve_scale(y="independent")

    display(chart)

In [ ]:
filtered_route(df)

#### Actual Faceted Charts (from Tiffany)
* Issues 
    * Faceting doesn't  with daily scheduled trip and average speed.

In [ ]:
def base_route_chart(df: pd.DataFrame, y_col: str) -> alt.Chart:
    """ """
    df = df.assign(
        time_period=df.time_period.str.replace("_", " ").str.title()
    ).reset_index(drop=True)

    selected_colors = [
        cp.CALITP_CATEGORY_BRIGHT_COLORS[0],  # blue
        cp.CALITP_CATEGORY_BRIGHT_COLORS[3],  # green
        cp.CALITP_CATEGORY_BOLD_COLORS[1],  # orange,
    ]

    # https://stackoverflow.com/questions/26454649/python-round-up-to-the-nearest-ten

    if "pct" in y_col:
        max_y = 1.2
    elif "per_minute" in y_col:
        max_y = round(df[y_col].max())
    else:
        max_y = round(df[y_col].max(), -1)

    df[y_col] = df[y_col].fillna(0).astype(int)

    chart = (
        alt.Chart(df)
        .mark_line()
        .encode(
            x=alt.X(
                "yearmonthdate(service_date):O",
                title="Date",
                axis=alt.Axis(format="%b %Y"),
            ),
            y=alt.Y(
                f"{y_col}:Q", title=labeling(y_col), scale=alt.Scale(domain=[0, max_y])
            ),
            color=alt.Color(
                "time_period:N",
                title=labeling("time_period"),
                scale=alt.Scale(range=cp.CALITP_SEQUENTIAL_COLORS),
            ),
            tooltip=[
                "route_combined_name",
                "route_id",
                "direction_id",
                "time_period",
                y_col,
            ],
        )
        .facet(
            column=alt.Column("direction_id:N", title=labeling("direction_id")),
        )
        .interactive()
    ).properties(title=labeling(y_col))

    return chart

In [ ]:
def filtered_route_charts(
    df: pd.DataFrame,
    control_field: str,
) -> alt.Chart:
    """
    https://stackoverflow.com/questions/58919888/multiple-selections-in-altair
    """
    route_dropdown = alt.binding_select(
        options=sorted(df[control_field].unique().tolist()),
        name="Routes ",
    )

    # Column that controls the bar charts
    route_selector = alt.selection_point(
        fields=[control_field],
        bind=route_dropdown,
    )

    sched_df = df[df.sched_rt_category != "vp_only"]
    vp_df = df[df.sched_rt_category != "schedule_only"]

    sched_service_chart = (
        base_route_chart(
            sched_df[sched_df.time_period == "all_day"], "avg_scheduled_service_minutes"
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    sched_trips_chart = (
        base_route_chart(sched_df, "n_scheduled_trips")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    sched_freq_chart = (
        base_route_chart(sched_df, "frequency")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    speeds_chart = (
        base_route_chart(vp_df, "speed_mph")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    ping_density_chart = (
        base_route_chart(vp_df, "vp_per_minute")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    spatial_accuracy_chart = (
        base_route_chart(vp_df, "pct_in_shape")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    atleast1vp_chart = (
        base_route_chart(vp_df, "pct_rt_journey_atleast1_vp")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    atleast2vp_chart = (
        base_route_chart(vp_df, "pct_rt_journey_atleast2_vp")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    chart_list = [
        sched_service_chart,
        sched_trips_chart,
        sched_freq_chart,
        speeds_chart,
        ping_density_chart,
        spatial_accuracy_chart,
        atleast1vp_chart,
        atleast2vp_chart,
    ]

    chart = alt.vconcat(*chart_list).resolve_scale(y="independent")

    return chart

In [ ]:
vp_df.columns

In [ ]:
filtered_route_charts(df, "route_combined_name")

#### Fake Faceted Chart  Doesn't work. The dropdowns don't impact the charts.

In [ ]:
def fake_faceted_chart2(df: pd.DataFrame, column_of_interest: str, title=str):

    ruler = (
        alt.Chart(df)
        .mark_rule(color="red", strokeDash=[10, 7])
        .encode(y=f"mean({column_of_interest}):Q")
    )

    dir_0 = df.loc[df.direction_id == 0]
    dir_1 = df.loc[df.direction_id == 1]
    chart_dir_0 = (
        alt.Chart(
            dir_0,
            title=alt.Title(
                title,
                subtitle="Direction 0",
            ),
        )
        .mark_bar()
        .encode(
            x=alt.X("month_year", title=labeling("month_year")),
            xOffset="time_period:N",
            y=alt.Y(f"{column_of_interest}:Q", title=labeling(column_of_interest)),
            color=alt.Color(
                "time_period:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
        )
        .properties(width=300, height=300)
    )
    chart_dir_1 = (
        alt.Chart(
            dir_1,
            title=alt.Title(
                "Title",
                color="white",
                subtitle="Direction 1",
            ),
        )
        .mark_bar()
        .encode(
            x=alt.X("month_year", title=labeling("month_year")),
            xOffset="time_period:N",
            y=alt.Y(f"{column_of_interest}:Q", axis=None),
            color=alt.Color(
                "time_period:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
        )
        .properties(width=300, height=300)
    )
    chart_dir_0 = chart_dir_0 + ruler
    chart_dir_1 = chart_dir_1 + ruler
    final_chart = alt.hconcat(chart_dir_0, chart_dir_1)
    return final_chart

In [ ]:
def fake_faceted_chart(
    df: pd.DataFrame,
    control_field: str,
) -> alt.Chart:
    """
    https://stackoverflow.com/questions/58919888/multiple-selections-in-altair
    """
    dropdown_list = df["route_combined_name"].unique().tolist()
    input_dropdown = alt.binding_select(options=dropdown_list, name="Routes")
    selection = alt.selection_point(fields=["route_combined_name"], bind=input_dropdown)

    selection = alt.selection_point(fields=["route_combined_name"], bind=input_dropdown)
    sched_df = df[df.sched_rt_category != "vp_only"]
    vp_df = df[df.sched_rt_category != "schedule_only"]

    daily_trips = fake_faceted_chart2(
        sched_df, "n_scheduled_trips", "Total Scheduled Daily Trips"
    )
    frequency = fake_faceted_chart2(
        sched_df, "frequency", "Frequency of Trips per Hour"
    )
    avg_speeds = fake_faceted_chart2(vp_df, "speed_mph", "Average Speed")

    chart_lists = [daily_trips, frequency, avg_speeds]
    chart = (
        alt.vconcat(*chart_lists)
        .resolve_scale(y="independent")
        .add_params(selection)
        .transform_filter(selection)
    )
    display(chart)

In [ ]:
# fake_faceted_chart(df, "route_combined_name")